# Global-Power-Plant-Database-Project

### Problem Statement:

#### Description

The Global Power Plant Database is a comprehensive, open source database of power plants around the world. It centralizes power plant data to make it easier to navigate, compare and draw insights for one’s own analysis. The database covers approximately 35,000 power plants from 167 countries and includes thermal plants (e.g. coal, gas, oil, nuclear, biomass, waste, geothermal) and renewables (e.g. hydro, wind, solar). Each power plant is geolocated and entries contain information on plant capacity, generation, ownership, and fuel type. It will be continuously updated as data becomes available.

In [ ]:
# Importing Required Libraries
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")
import joblib

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import zscore
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/wri/global-power-plant-database/master/source_databases_csv/database_IND.csv")

We have copied the raw file from the githib link and created a csv file in our system and imported the dataframe using pandas.

In [ ]:
df

In [ ]:
df.columns

<b>The dataset has 907 rows and 27 columns

We need to predict both capacity_mw (Continuous Target Variable) and Fuel Type (Categorical Target Variable) on seperate Regression and Classification Models.

In [ ]:
# Rename primary_fuel as Fuel_Type to understand the dataset in a easy format
df.rename(columns={'primary_fuel':'Fuel_Type'},inplace=True)

# Exploratory Data Analysis

In [ ]:
#Find the shape of database
df.shape

<b>We have 907 rows and 27 columns

In [ ]:
df.info()

<b> The database has null values and also object datatype.

In [ ]:
#Find the data types
df.dtypes

<b>The features that needs encoding are country, country_long, name, gppd_idnr,Fuel_Type,other_fuel1,other_fuel2, owner, source, url, geolocation_source,generation_data_source as they are object data type and the ML model needs numeric datatype.

In [ ]:
#Find the null values
df.isnull().sum()

<b>There are multiple null values in the datasets.

In [ ]:
# Find unique values
df.nunique()

<b>Here the columns country, country_long, other_fuel2, year_of_capacity_data and generation_data_source have only one unique value. Also other_fuel3, wepp_id,generation_gwh_2013, generation_gwh_2019, estimated_generation_gwh have no unique values which means they are filled with only NAN values. Since these columns have same entries thoughout the dataset so we can drop these columns.

# Feature Selection

country, country_long, other_fuel2, year_of_capacity_data and generation_data_source

other_fuel3, wepp_id,generation_gwh_2013, generation_gwh_2019, estimated_generation_gwh

In [ ]:
# Checking the count of the country
df["country"].value_counts()

<b>We see that only IND is listed in the country hence there is no impact of this feature on prediction

In [ ]:
# Checking the count of the country_long
df["country_long"].value_counts()

<b>We see that only India is listed in the country_long hence there is no impact of this feature on prediction

In [ ]:
df.drop(columns=["country","country_long"],axis=1,inplace=True)

In [ ]:
# Checking the count of other_fuel2
df["other_fuel2"].value_counts()

<b>We see that only Oil is listed in the other_fuel2 and rest are all 906 null values hence there is no impact of this feature on prediction

In [ ]:
# Checking the count of year_of_capacity_data
df["year_of_capacity_data"].value_counts()

<b>year_of_capacity_data has only 2019 listed value and 388 null values hence there is no impact of this feature on prediction

In [ ]:
# Checking the count of generation_data_source
df["generation_data_source"].value_counts()

<b>generation_data_source has only Central Electricity Authority as listed value and 458 null values hence there is no impact of this feature on prediction

In [ ]:
# Checking the count of other_fuel3
df["other_fuel3"].value_counts()

<b>other_fuel3 has all values as null values hence dropping this column

In [ ]:
# Checking the count of wepp_id
df["wepp_id"].value_counts()

<b>wepp_id has all values as null values hence dropping this column

In [ ]:
# Checking the count of generation_gwh_2013
df["generation_gwh_2013"].value_counts()

<b>generation_gwh_2013 has all values as null values hence dropping this column

In [ ]:
# Checking the count of generation_gwh_2019
df["generation_gwh_2019"].value_counts()

<b>generation_gwh_2019 has all values as null values hence dropping this column

In [ ]:
# Checking the count of generation_gwh_2019
df["estimated_generation_gwh"].value_counts()

<b>estimated_generation_gwh has all values as null values hence dropping this column

In [ ]:
df.drop(columns=["other_fuel2","year_of_capacity_data","generation_data_source","other_fuel3","wepp_id","generation_gwh_2013","generation_gwh_2019","estimated_generation_gwh"],axis=1,inplace=True)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
# Checking the count of name
df["name"].value_counts()

In [ ]:
# Checking the count of gppd_idnr
df["gppd_idnr"].value_counts()

In [ ]:
# Checking the count of owner
df["owner"].value_counts()

In [ ]:
# Checking the count of url
df["url"].value_counts()

<b>The columns "name", "gppd_idnr", "owner", "url" are all unique values and there is no impact of this feature on prediction hence we can drop these features

In [ ]:
df.drop(columns=["name","gppd_idnr","owner","url"],axis=1,inplace=True)

In [ ]:
df

In [ ]:
df.shape

In [ ]:
# Checking null values again after feature selection
df.isnull().sum()

In [ ]:
# Let's visualize the null values clearly
sns.heatmap(df.isnull())

We can clearly observe the white lines in the heat map which indicates the missing values in the dataset.

In [ ]:
#Checking the skewness of the dataset
df.skew().sort_values()

In [ ]:
# Checking the count of other_fuel1
df["other_fuel1"].value_counts()

## Treating null values using imputation techniques.

#### Checking the mean and mode values of the features having null values, so that we can fill the null values

In [ ]:
df.columns

In [ ]:
#checking the mean of latitude
df['latitude'].mean()

In [ ]:
#checking the mode of other_fuel1 
df["other_fuel1"].mode()

In [ ]:
#checking the mode of geolocation_source columns
df["geolocation_source"].mode()

#### Filling the null values

In [ ]:
df["latitude"] = df["latitude"].fillna(df["latitude"].mean())
df["other_fuel1"] = df["other_fuel1"].fillna(df["other_fuel1"].mode()[0])
df["geolocation_source"] = df["geolocation_source"].fillna(df["geolocation_source"].mode()[0])
df["longitude"] = df["longitude"].fillna(df["longitude"].median())
df["commissioning_year"] = df["commissioning_year"].fillna(df["commissioning_year"].median())
df["generation_gwh_2014"] = df["generation_gwh_2014"].fillna(df["generation_gwh_2014"].median())
df["generation_gwh_2015"] = df["generation_gwh_2015"].fillna(df["generation_gwh_2015"].median())
df["generation_gwh_2016"] = df["generation_gwh_2016"].fillna(df["generation_gwh_2016"].median())
df["generation_gwh_2017"] = df["generation_gwh_2017"].fillna(df["generation_gwh_2017"].median())
df["generation_gwh_2018"] = df["generation_gwh_2018"].fillna(df["generation_gwh_2018"].median())

In [ ]:
# checking for missing values after imputation.
df.isnull().sum()

Hence we have treated the null values now and the data now shows no null values

In [ ]:
# Heatmap
sns.heatmap(df.isnull())

Clearly there is no null values

In [ ]:
#Getting the columns in the dataset
df.columns

In [ ]:
# Checking the list of counts of capacity_mw
df['capacity_mw'].value_counts()

In [ ]:
# Checking the list of counts of primary_fuel
df['Fuel_Type'].value_counts()

In [ ]:
# Checking the uniqueness of primary_fuel
df["Fuel_Type"].unique()

## Feature Extraction

In [ ]:
# Checking the list of counts of commissioning_year
df['commissioning_year'].value_counts()

In [ ]:
# Let's extract power plant age from commissioning year by subtracting it from the year 2018
df["Power_plant_age"] = 2018 - df["commissioning_year"]
df.drop(columns=["commissioning_year"], inplace = True)

I have extracted Power plant age from commissioning year and dropped commissioning year columns. From Power plant age we can get to know how old are the power plants.

In [ ]:
df.head()

## Statistical Summary

In [ ]:
df.describe()

Here we can see the statistical analysis of the dataset (numerical only)

We can observe that the count of the columns are same, which means the dataset is balanced. The minimum capacity of the power plant is zero and maximum in 4760 and there is huge difference in mean and standard deviation.From the difference between maximum and 75% percentile we can infer that there are huge outliers present in most of the columns, will remove them using appropriate methods before building our model.

In [ ]:
#checking the categorical columns
cat_col=[]
for i in df.dtypes.index:
    if df.dtypes[i]=='object':
        cat_col.append(i)
print(cat_col)

In [ ]:
#checking the numeric columns for visualization
num_col=[]
for i in df.dtypes.index:
    if df.dtypes[i]!='object':
        num_col.append(i)
print(num_col)      

# Data Visualization

In [ ]:
df.columns

## Univariate Analysis

### Categorical column visualization

In [ ]:
print(df['Fuel_Type'].value_counts())   #visualizing the fuel types in Fuel_Type
plt.figure(figsize=(5,5))
sns.countplot(df['Fuel_Type'])
plt.show()

In the above count plot for "primary_fuel" column we can see that the highest number of values have been covered by coal and hydro fuel types then comes solar and wind. Finally we see that gas, biomass, oil and nuclear have very low data counts.

However when we will be considering "primary_fuel" as our target label then this is impose a class imbalance issue while trying to create a classification model and therefore will need to be treated accordingly.

In [ ]:
#checking the count of fuel1
print(df['other_fuel1'].value_counts())
plt.figure(figsize=(5,5))
sns.countplot(df['other_fuel1'])
plt.show()

It can be observed that 'other_fuel1' type has 3 unique types namely: 'Oil', 'Cogeneration other fuel', 'Gas'. And it is clearly seen that oil is the max used fuel type.

In [ ]:
# Visualizing the counts of owner
print(df["geolocation_source"].value_counts())
labels='WRI','Industry About','National Renewable Energy Laboratory'
plt.figure(figsize=(8,5))
sns.countplot(df['geolocation_source'])
plt.show()

Here it can be seen that the count of WRI is the max, which means that the max information is shared by this source.

In [ ]:
print(df['capacity_mw'].value_counts())   #visualizing the capacity_mw
plt.figure(figsize=(10,10))
sns.countplot(df['capacity_mw'])
plt.show()

Here it can be seen the counts withrespect to capacity_mw.

In [ ]:
plt.figure(figsize=(15,7))
values = list(df['Power_plant_age'].unique())
diag = sns.countplot(df["Power_plant_age"], palette="prism")
diag.set_xticklabels(labels=values, rotation=90)
plt.title("Year of power plant operation details\n")
plt.xlabel("List of years weighted by unit-capacity when data is available")
plt.ylabel("Count of Rows in the Dataset")
plt.show()

In the above count plot we can see the list of years as to when the power plant data was made available. Since we had missing values in the "commissioning_year" column we replaced it with the median wherein the year "15" covered the most rows in our dataset compared to all the other years.

### Checking the Distribution of the Dataset, if it is normal

Numerical Column

In [ ]:
# Checking how the data has been distriubted in each column

plt.figure(figsize=(10,10),facecolor='white')
plotnumber=1
for column in num_col:
    if plotnumber<=9:
        ax=plt.subplot(3,3,plotnumber)
        sns.distplot(df[column],color="b")
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.tight_layout()

Here in the plots we can see that the data is not normally distributed. Outliers and skewness is present, which needs to be treated.

## Bivariate Analysis

### Correlation between features and target 'Capacity_mw'

In [ ]:
#Checking the relation between target capacity_mw and variable geolocation source
plt.figure(figsize=[10,6])
plt.style.use('ggplot')
plt.title('Comparision between geolocation_source and capacity_mw')
sns.scatterplot(df['geolocation_source'],df["capacity_mw"])

Here also we can see that WRI 'geolocation_source' plays a major role

In [ ]:
#Checking the relation between power plant age and capacity_mw
plt.figure(figsize=[10,6])
plt.style.use('ggplot')
plt.title('Comparision between Power_plant_age and capacity_mw')
sns.scatterplot(df['Power_plant_age'],df["capacity_mw"])

Here we can see a negative correlation

In [ ]:
# Checking the relation between feature latitude and targer capacity_mw
plt.figure(figsize=[10,6])
plt.style.use('ggplot')
plt.title('Comparision between latitude and capacity_mw')
sns.scatterplot(df['latitude'],df["capacity_mw"])

Here this feature does not show any linear relationship

In [ ]:
# Checking the relationship between target longitude and capacity_mw
plt.figure(figsize=[10,6])
plt.style.use('ggplot')
plt.title('Comparision between longitude and capacity_mw')
sns.regplot(df['longitude'],df["capacity_mw"]);

This feature also does not show any linear relationship

In [ ]:
fig,axes=plt.subplots(3,2,figsize=(15,12))

#Checking the relation between feature generation_gwh_2013 and targer capacity_mw
sns.scatterplot(x = "generation_gwh_2014", y = "capacity_mw",ax=axes[0,0],data = df,color="b")

#Checking the relation between feature generation_gwh_2014 and targer capacity_mw
sns.scatterplot(x='generation_gwh_2015',y='capacity_mw',ax=axes[0,1],data=df,color="b")

#Checking the relation between feature generation_gwh_2015 and targer capacity_mw
sns.scatterplot(x='generation_gwh_2016',y='capacity_mw',ax=axes[1,0],data=df,color="b")

#Checking the relation between feature generation_gwh_2016 and targer capacity_mw
sns.scatterplot(x='generation_gwh_2017',y='capacity_mw',ax=axes[1,1],data=df,color="b")

#Checking the relation between feature generation_gwh_2017 and targer capacity_mw
sns.scatterplot(x='generation_gwh_2018',y='capacity_mw',ax=axes[2,0],data=df,color="b")
plt.show()

This features shows a positive correlation. Here the electricity generation reported for the years has capacity above 1000 mw also as the generation growth increases, the capacity of plant is also increasing moderately.

### Correlation between features and target 'Fuel_Types'

In [ ]:
#Checking the relation between target fuel_type and variable Power_plant_age
plt.figure(figsize=[10,6])
plt.title('Comparision between Power_plant_age and Fuel_Type')
sns.barplot(df['Power_plant_age'],df["Fuel_Type"])

Here we can see that older power plants uses Hydro as energy source, followed by oil. The newer power plants are using more of Coal, Solar and Gas

In [ ]:
# Checking the relation between feature latitude and targer Fuel_Type
plt.figure(figsize=[10,6])
plt.title('Comparision between latitude and Fuel_Type')
sns.barplot(df['latitude'],df["Fuel_Type"])

Solar has the highest latitude

In [ ]:
# Checking the relationship between target longitude and Fuel_Type
plt.figure(figsize=[10,6])
plt.title('Comparision between longitude and Fuel_Type')
sns.barplot(df['longitude'],df["Fuel_Type"]);

Here Gas shows the highest longitude

In [ ]:
fig,axes=plt.subplots(3,2,figsize=(15,12))

#Checking the relation between feature generation_gwh_2013 and targer Fuel_Type
sns.barplot(x = "generation_gwh_2014", y = "Fuel_Type",ax=axes[0,0],data = df,color="b")

#Checking the relation between feature generation_gwh_2014 and targer Fuel_Type
sns.barplot(x='generation_gwh_2015',y='Fuel_Type',ax=axes[0,1],data=df,color="b")

#Checking the relation between feature generation_gwh_2015 and targer Fuel_Type
sns.barplot(x='generation_gwh_2016',y='Fuel_Type',ax=axes[1,0],data=df,color="b")

#Checking the relation between feature generation_gwh_2016 and targer Fuel_Type
sns.barplot(x='generation_gwh_2017',y='Fuel_Type',ax=axes[1,1],data=df,color="b")

#Checking the relation between feature generation_gwh_2017 and targer Fuel_Type
sns.barplot(x='generation_gwh_2018',y='Fuel_Type',ax=axes[2,0],data=df,color="b")
plt.show()

Here we can see that the most used energy source in all the years is nuclear followed by coal

### Checking the relationship between both the targets

In [ ]:
plt.figure(figsize = (10,6))
plt.title("Comparision between Fuel Type and capacity_mw")
sns.barplot(x = "Fuel_Type", y = "capacity_mw", data = df)
plt.show()

Here also it shows that energy source Nuclear has the major contribution

## Label Encoding

In [ ]:
categorical_col = ['Fuel_Type', 'other_fuel1', 'source', 'geolocation_source']

In [ ]:
LE=LabelEncoder()
df[categorical_col]= df[categorical_col].apply(LE.fit_transform)

In [ ]:
df[categorical_col]

Now we have encoded the categorical columns

In [ ]:
df

## Identifying the outliers

In [ ]:
plt.figure(figsize=(10,10),facecolor='white')
plotnumber=1
for column in num_col:
    if plotnumber<=9:
        ax=plt.subplot(3,3,plotnumber)
        sns.boxplot(df[column],color="blue")
        plt.xlabel(column,fontsize=12)
    plotnumber+=1
plt.tight_layout()

In the boxplot we can notice the outliers present in all the columns except latitude. Even target column has outliers but no need to remove it. Let's remove outliers using Zscore method.

In [ ]:
df.columns

In [ ]:
# Features containing outliers
features = df[['longitude', 'generation_gwh_2014', 'generation_gwh_2015', 'generation_gwh_2016', 'generation_gwh_2017', 'generation_gwh_2018','Power_plant_age']]

In [ ]:
z=np.abs(zscore(features))

z

In [ ]:
# Creating new dataframe
new_df = df[(z<3).all(axis=1)] 
new_df

In [ ]:
df.shape

In [ ]:
new_df.shape

In [ ]:
print("total_dropped_rows",df.shape[0] - new_df.shape[0])

### Percentage data loss:

In [ ]:
loss_percent=(907-851)/907*100
print(loss_percent,'%')

checking the data loss percentage by comparing the rows in our original data set and the new data set after removal of the outliers. usually less than 10% data loss is acceptable

#### df_new is the new data set after all the unnecessary columns and all the outliers apart from target columns (with z<3 z score) are removed

## Correlation between the target variable and features

In [ ]:
cor = new_df.corr()
cor

In [ ]:
plt.figure(figsize=(25,22))
sns.heatmap(new_df.corr(),linewidths=.1,vmin=-1, vmax=1, fmt='.2g', annot = True, linecolor="black",annot_kws={'size':15},cmap="YlGnBu")
plt.yticks(rotation=0)

In [ ]:
new_df.corr()['Fuel_Type'].sort_values()

In [ ]:
new_df.corr()['capacity_mw'].sort_values()

Here we can see the co-relation between all the features and the features and targets

The label capacity_mw is highly positively correlated with the features generation_gwh_2017, generation_gwh_2016, generation_gwh_2015, generation_gwh_2014, generation_gwh_2013. And the label is negatively correlated with the features Fuel_Type, source and Power_plant_age. The columns other_fuel1 and latitude have no relation with the label, so we can drop them.

The label Fuel_Type is less correlated with Power_plant_age and source. The label is negatively correlated with geolocation_source, longitude, capacity_mw, and all generation_gwh years.

From the heat map we can notice most of the features are highly correlated with each other which leads to multicollinearity problem. So will try to solve this problem by Checking VIF value before building our models.

Also the features other_fuel1 and latitude have very very less correlation with both the labels. Hence after checking VIF we can think about dropping these 2 columns.

## Visualizing the correlation between label and features using bar plot

In [ ]:
plt.figure(figsize=(22,7))
new_df.corr()['Fuel_Type'].sort_values(ascending=False).drop(['Fuel_Type']).plot(kind='bar',color='c')
plt.xlabel('Feature',fontsize=10)
plt.ylabel('target',fontsize=10)
plt.title('correlation between label and feature using bar plot',fontsize=20)
plt.show()

## MultiCollinearity with Variance Inflation Factor

In [ ]:
df1=pd.DataFrame(data=new_df)       # copying the dataframe
df1

In [ ]:
x1=df1.iloc[:,1:]
y1=df1.iloc[:,0]

In [ ]:
x1

In [ ]:
y1

In [ ]:
x1.shape

In [ ]:
y1.shape

In [ ]:
x1.shape[1]    # 12 number of columns

In [ ]:
# importing required libraries
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
def calc_vif(x1):
    vif=pd.DataFrame()
    vif["variables"]=x1.columns
    vif["VIF FACTOR"]=[variance_inflation_factor(x1.values,i) for i in range(x1.shape[1])]
    return(vif)

In [ ]:
calc_vif(x1)

other_fuel1 has the highest VIF FACTOR hence lets drop it first

In [ ]:
x1=df1.drop(['other_fuel1'],axis=1)

In [ ]:
calc_vif(x1)

Since latitude has the lowest contribution compared to both the targets lets drop that first and see what happens

In [ ]:
x1=df1.drop(['other_fuel1','latitude'],axis=1)

In [ ]:
calc_vif(x1)

We can see VIF has decreased drastically cause of removing of these 2 columns. Hence lets drop these 2 columns next

## Feature selection by dropping columns

In [ ]:
new_df.drop("other_fuel1",axis=1,inplace=True)
new_df.drop("latitude",axis=1,inplace=True)

In [ ]:
new_df.head()

In [ ]:
sns.pairplot(new_df)

We can see the relation between all the features and target variable using pairplot.

# Machine Learning

## 1.Predicting "Capacity_mw" Target

### Splitting the dataset into Features and Target

In [ ]:
x=new_df.drop('capacity_mw', axis=1)
y=new_df["capacity_mw"]

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x

In [ ]:
y

### Checking for skewness

In [ ]:
x.skew().sort_values()

The following columns have skewness more than +0.5 and -0.5.

longitude generation_gwh_2013 generation_gwh_2014 generation_gwh_2015 generation_gwh_2016 generation_gwh_2017 Power_plant_age

### Removing skewness using yeo-johnson method

In [ ]:
from sklearn.preprocessing import PowerTransformer
skew = ['longitude','generation_gwh_2014','generation_gwh_2015','generation_gwh_2016','generation_gwh_2017','generation_gwh_2018','Power_plant_age']
transf = PowerTransformer(method='yeo-johnson')

In [ ]:
x[skew] = transf.fit_transform(x[skew].values)
x[skew].head()

In [ ]:
x.skew()

Since Fuel_Type, source and geolocation_source were categorically encoded values we didnt use transformation for skewness removal.

Rest of the numerical data columns the skewness has been removed.

In [ ]:
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in x[skew]:
    if plotnumber<=9:
        ax = plt.subplot(3,3,plotnumber)
        sns.distplot(x[column],color='g',kde_kws={"shade": True},hist=False)
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

The dataset looks normal now

## Feature Scalling

In [ ]:
#Scalling the data using Standard Scaler
scaler = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)
x

The dataset x has now been scaled.

### MultiCollinearity with Variance Inflation Factor

In [ ]:
vif = pd.DataFrame()
vif["VIF values"] = [variance_inflation_factor(x.values,i)
              for i in range(len(x.columns))]
vif["Features"] = x.columns

# Let's check the values
vif

VIF values in all the columns are less then 10, hence no multicolinearity problem exists.

### Finding best random state

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
#getting the best random state for .20 test size
maxAccu=0
maxRS=0
for i in range(1,200): 
    x_train,x_test, y_train, y_test =train_test_split(x,y, test_size=.20,random_state=i)
    mod=RandomForestRegressor()
    mod.fit(x_train, y_train)
    pred=mod.predict(x_test)
    acc=r2_score(y_test,pred)
    if acc>maxAccu:
        maxAccu=acc
        maxRS=i
print('R2 Score=', maxAccu, 'Random_State',maxRS)

In [ ]:
#getting the best random state for .30 test size
maxAccu=0
maxRS=0
for i in range(1,200): 
    x_train,x_test, y_train, y_test =train_test_split(x,y, test_size=.30,random_state=i)
    mod=RandomForestRegressor()
    mod.fit(x_train, y_train)
    pred=mod.predict(x_test)
    acc=r2_score(y_test,pred)
    if acc>maxAccu:
        maxAccu=acc
        maxRS=i
print('R2 Score=', maxAccu, 'Random_State',maxRS)

We got best r2 score of 0.878 at a random state of 125 for test_size=.20

### Train_test_Split

In [ ]:
x_train,x_test, y_train, y_test=train_test_split(x,y,test_size=.20, random_state=125)

In [ ]:
# importing all the required libraries

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# creating a function to run all the regressors

def regressor(model, x, y):
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=125)
    
    # Training the model
    model.fit(x_train, y_train)
    
    # Predicting y_test
    pred = model.predict(x_test)
    
    # Root Mean Square Error (RMSE)
    rmse = mean_squared_error(y_test, pred, squared=False)
    print("Root Mean Square Error is:", rmse)
    
    # R2 score
    r2 = r2_score(y_test, pred, multioutput='variance_weighted')*100
    print("R2 Score is:", r2)
    
    # Cross Validation Score
    cv_score = (cross_val_score(model, x, y, cv=5).mean())*100
    print("Cross Validation Score is:", cv_score)
    
    # Result of r2 score - cv score
    result = r2 - cv_score
    print("R2 Score - Cross Validation Score is", result)

### Linear Regression

In [ ]:
model=LinearRegression()
regressor(model, x, y)

### L1 -- Lasso Regression

In [ ]:
model=Lasso(alpha=0.001)
regressor(model, x, y)

### L2 -- Ridge Regression

In [ ]:
model=Ridge(alpha=0.001)
regressor(model, x, y)

### Elastic Net

In [ ]:
model=ElasticNet(alpha=0.001)
regressor(model, x, y)

### Support Vector Regression

In [ ]:
model=SVR(kernel='rbf')
regressor(model, x, y)

In [ ]:
model=SVR(kernel='poly')
regressor(model, x, y)

In [ ]:
model=SVR(kernel='linear')
regressor(model, x, y)

### Decision Tree Regressor

In [ ]:
model=DecisionTreeRegressor(random_state=125)
regressor(model, x, y)

In [ ]:
model=DecisionTreeRegressor()
regressor(model, x, y)

### Random Forest Regressor

In [ ]:
model=RandomForestRegressor()
regressor(model, x, y)

In [ ]:
model=RandomForestRegressor(random_state=125)
regressor(model, x, y)

### K Neighbors Regressor

In [ ]:
model=KNeighborsRegressor()
regressor(model, x, y)

### SGD Regressor

In [ ]:
model=SGDRegressor()
regressor(model, x, y)

### Gradient Boosting Regressor

In [ ]:
model=GradientBoostingRegressor()
regressor(model, x, y)

### Ada Boost Regressor

In [ ]:
model=AdaBoostRegressor(random_state=125)
regressor(model, x, y)

In [ ]:
model=AdaBoostRegressor()
regressor(model, x, y)

### Extra Trees Regressor

In [ ]:
model=ExtraTreesRegressor(random_state=125)
regressor(model, x, y)

In [ ]:
model=ExtraTreesRegressor()
regressor(model, x, y)

Comparing all the above the Extra Trees Regressor gives the best results since the R2 Score - Cross Validation Score are closest along with higher Cross Validation Score and the highest R2 score comparing all the models.

# Hyper parameter tuning

In [ ]:
#ExtraTreesRegressor?

In [ ]:
ExtraTreesRegressor().get_params().keys()

In [ ]:
# creating parameters list to pass into GridSearchCV

parameters = {'criterion' : ['squared_error', 'absolute_error'],
              'max_features' : ['auto', 'sqrt', 'log2'],
              'n_jobs' : [5, 10, 15]}

In [ ]:
GCV = GridSearchCV(ExtraTreesRegressor(), parameters, cv=5)

In [ ]:
GCV.fit(x_train,y_train)

In [ ]:
GCV.best_params_      # printing best parameters found by GridSearchCV

We got the best parameters using Gridsearch CV

In [ ]:
final_model = ExtraTreesRegressor(criterion = 'absolute_error', max_features = 'log2', n_jobs = 15)

In [ ]:
final_fit = final_model.fit(x_train,y_train)   # final fit

In [ ]:
final_pred = final_model.predict(x_test)   # predicting with best parameters

In [ ]:
best_r2=r2_score(y_test,final_pred,multioutput='variance_weighted')*100   # checking final r2_score
print("R2 score for the Best Model is:", best_r2)

In [ ]:
final_cv_score = (cross_val_score(final_model, x, y, cv=5).mean())*100
print("Cross Validation Score is:", final_cv_score)

In [ ]:
final_rmse = mean_squared_error(y_test, final_pred, squared=False)
print("Root Mean Square Error is:", final_rmse)

We used Hyper Parameter Tuning on the final model to obtained the best r2_score and CV score.

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x=y_test, y=final_pred, color='r')
plt1 = max(max(final_pred), max(y_test))
plt2 = min(min(final_pred), min(y_test))
plt.plot([plt1, plt2], [plt1, plt2], 'b-')
plt.xlabel('Actual Capacity_mw', fontsize=14)
plt.ylabel('Predicted Capacity_mw', fontsize=14)
plt.title('ExtraTreesRegressor',fontsize=18)
plt.show()

Plotting the Final model Actual Capacity_mw vs Predicted Capacity_mw

Hence after Hyper Parameter Tuning on the final model to obtained the best r2_score 89.249% and CV score 79.4% and lowest Root Mean Square Error is: 127.93.

## Saving the model in pickle Format

In [ ]:
# pickeling or serialization of a file
import pickle
filename = 'Global_Power_Plant_Capacity_mw_Regression_final_model.pkl'
pickle.dump(final_model, open(filename, 'wb'))

Saving the best regression model using pickle

## Prediction Conclusion:

In [ ]:
import numpy as np
a=np.array(y_test)
predicted=np.array(final_model.predict(x_test))
df_comparison = pd.DataFrame({"original":a,"predicted":predicted},index= range(len(a)))
df_comparison

Hence predicted the Capacity_mw using the x_test feature columns.

In [ ]:
df_comparison.to_csv('Global_Power_Plant_Capacity_mw_Regression_Prediction.csv')

Saving the predicted values in a csv file

# 2. Predicting "Fuel_Type" Target

## Seperating the Dataset into Features and Label(Fuel_Type)

In [ ]:
x_df = new_df.drop("Fuel_Type", axis=1)
y_df = new_df["Fuel_Type"]

In [ ]:
x_df.shape

In [ ]:
y_df.shape

## Checking the Skewness of x_df

In [ ]:
x_df.skew().sort_values()

We can see that there are skewness in most of the columns

## Removing the skewness

In [ ]:
skew = ['capacity_mw','longitude','generation_gwh_2014','generation_gwh_2015','generation_gwh_2016','generation_gwh_2017','generation_gwh_2018','Power_plant_age']

from sklearn.preprocessing import PowerTransformer
transfo = PowerTransformer(method='yeo-johnson')

transforming all the numerical columns apart from categorically encoded columns

In [ ]:
x_df[skew] = transfo.fit_transform(x_df[skew].values)
x_df[skew].head()

In [ ]:
x_df.skew().sort_values()

In [ ]:
#Lets visualize the data

plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in x_df[skew]:
    if plotnumber<=9:
        ax = plt.subplot(3,3,plotnumber)
        sns.distplot(x_df[column],color='b',kde_kws={"shade": True},hist=False)
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

The dataset looks normally distributed now

## Feature Scaling

In [ ]:
scaler=StandardScaler()
x_df=pd.DataFrame(scaler.fit_transform(x_df),columns=x_df.columns)
x_df

We have scaled the dataset.

## Checking Multicolinearity

In [ ]:
vif = pd.DataFrame()
vif["VIF values"] = [variance_inflation_factor(x_df.values,i)
              for i in range(len(x_df.columns))]
vif["Features"] = x_df.columns

# Let's check the values
vif

All the columns has vif values less then 10, hence there is no multicolinearity that exist.

In [ ]:
y_df.value_counts()

We can see that the target Fuel_Type has multiple classes in the mode of energy source, hence we can see that this is a multi classification problem. As the data between the classes are not balanced with 1 having 238 counts and 4 having only 9 counts, we have to do SMOTE oversampling of the data.

## SMOTE OverSampling

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
x_df, y_df = sm.fit_resample(x_df,y_df)

In [ ]:
y_df.value_counts()

Here we can see that the data imbalance has been removed.

In [ ]:
X = x_df   # renaming the features variable

In [ ]:
X

In [ ]:
Y = y_df   # renaming the target variable

In [ ]:
Y

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report

## Getting the best random state

In [ ]:
maxAccu=0
maxRS=0

for i in range(1,200):
    x_train,x_test, y_train, y_test=train_test_split(X,Y,test_size=.20, random_state=i)
    rfc=RandomForestClassifier()
    rfc.fit(x_train,y_train)
    pred=rfc.predict(x_test)
    acc=accuracy_score(y_test,pred)
    if acc>maxAccu:
        maxAccu=acc
        maxRS=i
print("Best accuracy is ",maxAccu," on Random_state ",maxRS) 

Hence we get best accuracy score as 0.952 at Random_state  142 in RandomForestClassifier

## train_test_split

In [ ]:
x_train,x_test, y_train, y_test=train_test_split(X,Y,test_size=.20, random_state=142)

In [ ]:
# Importing required libraries
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# creating a function to run all the classifiers

def classifier(model, X, Y):
    x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.2, random_state=142) 
    
    # Training the model
    model.fit(x_train, y_train)
    
    # Predicting y_test
    pred = model.predict(x_test)
    
    # Accuracy Score
    acc_score = (accuracy_score(y_test, pred))*100
    print("Accuracy Score:", acc_score)
    
    # Classification Report
    class_report = classification_report(y_test, pred)
    print("\nClassification Report:\n", class_report)
    
    # Cross Validation Score
    cv_score = (cross_val_score(model, X, Y, cv=5).mean())*100
    print("Cross Validation Score:", cv_score)
    
    # Result of accuracy minus cv scores
    result = acc_score - cv_score
    print("\nAccuracy Score - Cross Validation Score is", result)

### Logistic Regression

In [ ]:
model = LogisticRegression()
classifier(model, X, Y)

### Naive Bayes

In [ ]:
model = GaussianNB()
classifier(model, X, Y)

### SVC Classifier

In [ ]:
model = SVC(kernel='rbf')
classifier(model, X, Y)

In [ ]:
model = SVC(kernel='linear')
classifier(model, X, Y)

In [ ]:
model = SVC(kernel='poly')
classifier(model, X, Y)

### Decision Tree Classifier

In [ ]:
model = DecisionTreeClassifier()
classifier(model, X, Y)

### KNeighbors Classifier

In [ ]:
model = KNeighborsClassifier()
classifier(model, X, Y)

### SGD Classifier

In [ ]:
model = SGDClassifier()
classifier(model, X, Y)

### Random Forest Classifier

In [ ]:
model = RandomForestClassifier(random_state=142)
classifier(model, X, Y)

In [ ]:
model = RandomForestClassifier()
classifier(model, X, Y)

### ExtraTrees Classifier

In [ ]:
model = ExtraTreesClassifier()
classifier(model, X, Y)

### AdaBoost Classifier

In [ ]:
model = AdaBoostClassifier()
classifier(model, X, Y)

### Gradient Boosting Classifier

In [ ]:
model = GradientBoostingClassifier()
classifier(model, X, Y)

Comparing all the above the ExtraTreesClassifier gives the best results since the Accuracy Score - Cross Validation Score is the least along with higher Cross Validation Score and the highest Accuracy Score comparing all the models.

# Hyper Parameter Tuning

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.20,random_state=142)

In [ ]:
#ExtraTreesClassifier?

In [ ]:
# creating parameters list to pass into GridSearchCV

parameters = {'criterion' : ['gini', 'entropy'],
              'max_features' : ['auto', 'sqrt', 'log2'],
              'n_jobs' : [5, 10, 15]}

In [ ]:
GCV = GridSearchCV(ExtraTreesClassifier(), parameters, cv=5)

In [ ]:
GCV.fit(x_train,y_train)

In [ ]:
GCV.best_params_      # printing best parameters found by GridSearchCV

We got the best parameters using Gridsearch CV

In [ ]:
final_modelc = ExtraTreesClassifier(criterion = 'gini', max_features = 'log2', n_jobs = 5)   # final model with best parameters

In [ ]:
final_fitc = final_modelc.fit(x_train,y_train)   # final fit

In [ ]:
final_predc = final_modelc.predict(x_test)   # predicting with best parameters

In [ ]:
best_acc_score = (accuracy_score(y_test, final_predc))*100    # checking accuracy score
print("The Accuracy Score for the Best Model is ", best_acc_score)

We successfully performed the Hyper Parameter Tuning on the Final Model.

In [ ]:
# Final Cross Validation Score
final_cv_score = (cross_val_score(final_modelc, X, Y, cv=5).mean())*100
print("Cross Validation Score:", final_cv_score)

We got final accuracy score of 94.488% and Cross Validation Score of 92.12% which is good

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
# Final Classification Report
final_class_report = classification_report(y_test, final_predc)
print("\nClassification Report:\n", final_class_report)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
classifier = OneVsRestClassifier(final_modelc)
y_score = classifier.fit(x_train, y_train).predict_proba(x_test)

#Binarize the output
y_test_bin  = label_binarize(y_test, classes=[0,1,2,3,4,5,6,7])
n_classes = 8

# Compute ROC curve and AUC for all the classes
false_positive_rate = dict()
true_positive_rate = dict()
roc_auc = dict()
for i in range(n_classes):
    false_positive_rate[i], true_positive_rate[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(false_positive_rate[i], true_positive_rate[i])
    
   
for i in range(n_classes):
    plt.plot(false_positive_rate[i], true_positive_rate[i], lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multiclassification data')
plt.legend(loc="lower right")
plt.show()

Hence we got AUC ROC curve for all 8 classes which is either 0.98 or 1 which is good

# Saving the model in pickle Format

In [ ]:
# pickeling or serialization of a file
import pickle
filenamec = 'Global_Power_Plant_Fuel_Type_Classification_final_model.pkl'
pickle.dump(final_modelc, open(filenamec, 'wb'))

# Prediction Conclusion:

In [ ]:
import numpy as np
ac=np.array(y_test)
predictedc=np.array(final_modelc.predict(x_test))
df_comparisonc = pd.DataFrame({"original":ac,"predicted":predictedc},index= range(len(ac)))
df_comparisonc

Hence predicted the 'Fuel_Type' using the x_test feature columns.

In [ ]:
df_comparisonc.to_csv('Global_Power_Plant_Fuel_Type_Classification_Prediction.csv')

Saving the predicted values in a csv file